<a href="https://colab.research.google.com/github/xin-2001/taica_1132_GenAI/blob/main/HW7/113_2GenAI_HW7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【Demo06b】RAG02_打造_RAG_系統
- 作業說明：
  - 準備一份自己練習的資料。
  - 參考老師程式碼，修改成自己的樣子。
  - 程式碼記得加上讀你的zip檔網址的程式(助教才能執行)。
  - [雲端轉成可下載的網址](https://sites.google.com/site/gdocs2direct/?fbclid=IwZXh0bgNhZW0CMTAAAR5ji3w5J7kt2m2qRX7KXWKUKrxKVIj5nqQP5xKQb7fa2hHp97uh2DfpVmO07Q_aem_QX9JsZTBwEnvmiXwQtpC0g)
  - Gradio展示。

## 下載檔案並解壓縮

In [15]:
!wget https://github.com/xin-2001/taica_1132_GenAI/raw/refs/heads/main/HW7/faiss_db.zip

--2025-04-14 16:59:28--  https://github.com/xin-2001/taica_1132_GenAI/raw/refs/heads/main/HW7/faiss_db.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/xin-2001/taica_1132_GenAI/refs/heads/main/HW7/faiss_db.zip [following]
--2025-04-14 16:59:28--  https://raw.githubusercontent.com/xin-2001/taica_1132_GenAI/refs/heads/main/HW7/faiss_db.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243128 (237K) [application/zip]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>] 237.43K  --.-KB/s    in 0.01s   

2025-04-14 16:59:28 (18.5 MB/s) - ‘faiss_db.zip’ saved [243128/243128]

In [16]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
   creating: faiss_db/
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


## 1. 安裝並引入必要套件

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.73.0
    Uninstalling openai-1.73.0:
      Successfully uninstalled openai-1.73.0


In [4]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [5]:
from openai import OpenAI
import gradio as gr

## 2. 自訂 E5 embedding 類別

In [17]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 3. 載入 `faiss_db`

In [18]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## 4. 設定 LLM

In [19]:
import os
from google.colab import userdata

如之前, 我們會用 OpenAI API。使用 Gemini 服務。

In [20]:
api_key = userdata.get('GOOGLE_API_KEY')

In [21]:
os.environ["OPENAI_API_KEY"] = api_key

In [22]:
model = "gemini-1.5-flash"
base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

In [23]:
client = OpenAI(
    base_url=base_url # 使用 OpenAI 本身不需要這段
)

### 5. `prompt` 設計

In [24]:
system_prompt = """
你是一位台灣教育部科技領域的108課綱負責人，負責推動與發展科技領域課程。你的任務是根據提供的資料（課綱內容）來回答使用者的問題，並協助他們理解課綱的精神與目標。

請以親切、簡潔的語言風格回答問題，並附帶具體的建議。

請用台灣習慣的中文回應。

在回答問題時，請注意以下幾點：

1. 務必根據提供的資料（課綱內容）來回答問題，避免提供個人意見或臆測。
2. 答案應盡量簡潔易懂，避免使用過於專業的術語或艱澀的詞彙。
3. 若資料不足以回答問題，請告知使用者並建議他們參考相關資源，例如教育部網站或課綱文件。
4. 請以台灣習慣的中文回應，避免使用中國大陸用語或英文縮寫。
5. 請展現親切、專業的態度，並盡力協助使用者解決問題。
"""

prompt_template = """
根據下列資料回答問題：
{retrieved_chunks}

使用者的問題是：{question}

請根據資料內容回覆，若資料不足請告訴同學可以詢問教育部。
"""

## 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [25]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history
    # 取回相關資料
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer

## 7. 用 Gradio 打造 Web App

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 📚 108課綱科技領域AI諮詢師\n 任何臺灣108課綱科技領域的問題都可以問我呦!")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...", label="輸入訊息")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

<ipython-input-28-bbef0704a708>:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8ab69fd7e3ce465d18.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 成果展現
- prompt設計
  > 你是一位台灣教育部科技領域的108課綱負責人，負責推動與發展科技領域課程。你的任務是根據提供的資料（課綱內容）來回答使用者的問題，並協助他們理解課綱的精神與目標。
  >
  > 請以親切、簡潔的語言風格回答問題，並附帶具體的建議。
  >
  > 請用台灣習慣的中文回應。
  >
  > 在回答問題時，請注意以下幾點：
  >
  > 1. 務必根據提供的資料（課綱內容）來回答問題，避免提供個人意見或臆測。
  > 2. 答案應盡量簡潔易懂，避免使用過於專業的術語或艱澀的詞彙。
  > 3. 若資料不足以回答問題，請告知使用者並建議他們參考相關資源，例如教育部網站或課綱文件。
  > 4. 請以台灣習慣的中文回應，避免使用中國大陸用語或英文縮寫。
  > 5. 請展現親切、專業的態度，並盡力協助使用者解決問題。
- 結果
  > ![image.png](https://github.com/xin-2001/taica_1132_GenAI/blob/main/HW7/hw7_2_1.png?raw=true)

## 評分標準：

0分：程式連結無法順利開啟，且無截圖。

1分：與老師範例一樣。

2分：GPT 水準或繳交作業與本周主題無關(若貼成別週作業也列在此。)

4分：程式連結無法順利開啟，但有部份截圖。

6分：主題與老師的範例相似。(校園社團資料)

7~8分：達成大致作業要求。

9分：達成作業要求。


繳交區沒打標題、重點說明、重點截圖會斟酌扣分。
主題有趣+1分。

- 註一：若沒有引入老師的固定套件，總分 -1。
- 註二：程式連結無法順利開啟包含。 1.權限未開啟 2.繳交非colab連結的檔案 3.程式碼無法完整執行
- 註三：有請生成式AI幫忙的地方請特別說明，加上理解後的說明，並附上截圖(包括Prompt與生成結果)，不然都當抄襲AI。
- 註四：若認定抄襲，除該次作業0分外，總成績-10，再犯再扣，直至總成績扣完為止。
- 註五：若覺得自己重點的東西很多，說明或截圖也可整理成pdf檔。